# Recommandations par lots à l'aide de l'algorithme de personnalisation de l'utilisateur AWS et des données d'interaction article-utilisateur <a class="anchor" id="top"></a>

Dans ce bloc-notes, vous allez choisir un jeu de données et le préparer à être utilisé avec les recommandations par lots d'Amazon Personalize.

1. [Choisir un jeu de données ou une source de données](#source)
1. [Préparer vos données](#prepare)
1. [Créer des groupes de jeux de données et le jeu de données d'interactions](#group_dataset)
1. [Configurer un compartiment S3 et un rôle IAM](#bucket_role)
1. [Importer les données d'interactions](#import)
1. [Créer des solutions](#solutions)
1. [Recommandations par lots](#batch)
1. [Nettoyer](#cleanup)

## Introduction <a class="anchor" id="intro"></a>

Pour la plupart, les algorithmes d'Amazon Personalize (appelés recettes) cherchent à résoudre différentes tâches, expliquées ici :

1. **Personnalisation de l'utilisateur AWS** – Recommande des articles en fonction des interactions précédentes de l'utilisateur avec les articles.
1. **Classement personnalisé** – Prend une collection d'articles et les ordonne dans l'ordre probable d'intérêt en utilisant une approche de type HRNN.
1. **SIMS (Éléments similaires)** – À partir d'un article donné, recommande d'autres articles avec lesquels les utilisateurs ont également interagi.
1. **Compte de popularité** – Recommande les articles les plus populaires, si HRNN ou métadonnées HRNN n'ont pas de réponse – cela est renvoyé par défaut.

Quel que soit le cas d'utilisation, les algorithmes partagent tous une base d'apprentissage sur les données d'interaction entre l'utilisateur et l'article, qui sont définies par trois attributs fondamentaux :

1. **UserID** – L'utilisateur qui a interagi
1. **ItemID** – Article avec lequel l'utilisateur a interagi
1. **Horodatage** – L'heure à laquelle l'interaction s'est produite

Nous prenons également en charge les types d'événements et les valeurs d'événements définis par :

1. **Type d'événement​​** – Étiquette catégorique d'un événement (parcourir, acheté, évalué, etc.).
1. **Valeur de l'événement** – Une valeur correspondant au type d'événement qui s'est produit. En règle générale, nous recherchons des valeurs normalisées entre 0 et 1 relatives aux types d'événements. Par exemple, s'il y a trois phases pour terminer une transaction (cliqué, ajouté au panier et acheté), alors il y aura une valeur event_value pour chaque phase comme 0,33, 0,66 et 1,0 respectivement.

Les champs type d'événement et valeur d'événement sont des données supplémentaires qui peuvent être utilisées pour filtrer les données envoyées en vue de l'entraînement du modèle de personnalisation. Dans cet exercice particulier, nous n'aurons pas de type d'événement ou de valeur d'événement. 

## Choisir un jeu de données ou une source de données <a class="anchor" id="source"></a>
[Retour au début](#top)

Comme nous l'avons mentionné, les données d'interaction utilisateur-article sont essentielles pour commencer à utiliser le service. Cela signifie que nous devons rechercher les cas d'utilisation qui génèrent ce type de données, dont voici quelques exemples courants :

1. Applications de vidéo à la demande
1. Plateformes d'e-commerce
1. Agrégateurs/plateformes de médias sociaux

Il existe quelques lignes directrices pour définir la portée d'un problème adapté à Personalize. Nous recommandons les valeurs ci-dessous comme point de départ, bien que les [limites officielles](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) soient un peu plus basses.

* Utilisateurs authentifiés
* Au moins 50 utilisateurs uniques
* Au moins 100 articles uniques
* Au moins deux douzaines d'interactions pour chaque utilisateur 

La plupart du temps, il est facile d'y parvenir, et si vous êtes dans une catégorie faible, vous pouvez souvent compenser en ayant un chiffre plus élevé dans une autre catégorie.

En règle générale, vos données n'arriveront pas sous une forme parfaite pour Personalize, et il faudra les modifier afin qu'elles soient structurées correctement. Ce bloc-notes a pour objectif de vous guider dans tout ceci. 

Pour commencer, nous allons utiliser le jeu de données [Last.FM](https://grouplens.org/datasets/hetrec-2011/). Il s'agit d'enregistrements du comportement d'écoute musicale de ses utilisateurs. Les données correspondent à nos directives avec un grand nombre d'utilisateurs, d'articles et d'interactions.

Tout d'abord, vous allez télécharger le jeu de données et le décompresser dans un nouveau dossier en utilisant le code ci-dessous.

In [ ]:
data_dir = "data"
!mkdir $data_dir
!cd $data_dir && wget http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
!cd $data_dir && unzip hetrec2011-lastfm-2k.zip

Examinez les fichiers de données que vous avez téléchargés.

In [ ]:
!ls $data_dir

Pour l'instant, nous avons très peu d'informations sur les données, si ce n'est qu'il semble exister un grand nombre de fichiers .dat et un fichier README. L'ouverture du fichier README nous renseignera sur la structure globale de ces données. C'est une étape que vous pouvez probablement ignorer avec des données personnalisées, sauf si la source de données provient d'une équipe externe

D'après le fichier README, nous constatons qu'il existe plusieurs types d'interactions dans ce jeu de données. Les interactions entre les utilisateurs qui se marquent mutuellement comme amis, les interactions des utilisateurs qui écoutent des artistes et les interactions des identifications attribuées aux utilisateurs et aux artistes.

Dans ce cas, nous nous concentrons sur les utilisateurs, les artistes et les interactions d'écoute. Nous avons 1 892 utilisateurs, 17 632 artistes (nos articles dans ce cas) et 92 834 interactions entre les utilisateurs et les artistes écoutés. Cela suffit amplement pour commencer à utiliser Personalize.

Continuez à lire le fichier README jusqu'à la section `Files`. La plupart des fichiers du jeu de données ne sont pas pertinents pour nous, mais ce fichier `users_artists.dat` semble prometteur. La section `Data format` du fichier README fournit plus de détails sur le contenu du fichier. C'est là que nous rencontrons notre premier problème.

| userID | artistID | poids  |
|--------|----------|---------|
| 2      | 51       | 13883   |

Bien qu'il existe des données d'interaction entre les utilisateurs et les artistes qu'ils écoutent, ces interactions sont stockées sous forme de poids et non d'horodatage. Nous avons besoin des données d'interaction utilisateur-article-horodatage pour Amazon Personalize. 

Si vous examinez à nouveau les fichiers dans le jeu de données, vous devez constater que `users_taggedartists-timestamps.dat` contient des données d'horodatage. Et si nous utilisions le comportement d'identification comme données d'interaction, au lieu du comportement d'écoute ? Pouvons-nous supposer qu'un utilisateur qui identifie un artiste est une indication d'un sentiment positif ? Normalement, vous discuterez avec votre client, ou avec quelqu'un qui a des connaissances dans le domaine, pour comprendre si cette interaction est adaptée au cas d'utilisation que vous voulez résoudre. Pour l'instant, nous supposerons que le comportement d'identification est adapté à nos besoins. 

Le schéma pour `user_taggedartists-timestamps.dat` est :

| userID | artistID | tagID | timestamp     |
|--------|----------|-------|---------------|
| 2      | 52       | 13    | 1238536800000 |

Si nous supprimons l'attribut `tagID`, nous avons exactement le format dont nous avons besoin pour Amazon Personalize.

## Préparer vos données <a class="anchor" id="prepare"></a>
[Retour au début](#top)

Chargez ensuite les données et assurez-vous qu'elles sont en bon état, puis enregistrez-les dans un CSV où elles sont prêtes à être utilisées avec Amazon Personalize.

Pour commencer, importez une collection de bibliothèques Python couramment utilisées en science des données.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime

import boto3
import pandas as pd

Ensuite, ouvrez le fichier de données et examinez les premières lignes.

In [ ]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat')
original_data.head(5)

À l'évidence, les données ne se sont pas chargées correctement. Le délimiteur par défaut des fichiers CSV (comma-separated value) est la virgule (`,`), mais dans ce cas, le fichier a été enregistré avec des caractères de tabulation (`\t`). Spécifions donc le délimiteur correct et essayons à nouveau de charger les données.

In [ ]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat', delimiter='\t')
original_data.head(5)

C'est mieux. Maintenant que les données ont été chargées en mémoire, extrayons quelques informations supplémentaires. Tout d'abord, calculez quelques statistiques de base à partir des données.

In [ ]:
original_data.describe()

Cela montre que nous avons une bonne gamme de valeurs pour `userID` et `artistID`. Ensuite, il est toujours bon de vérifier le format des données.

In [ ]:
original_data.info()

À partir de là, vous pouvez voir qu'il y a un total de 186 479 entrées dans le jeu de données, avec 4 colonnes, et que chaque cellule est stockée au format int64.

Le format int64 est clairement adapté à `userID` et `artistID`. Cependant, nous effectuer une analyse plus approfondie pour comprendre les horodatages dans les données. Pour utiliser Amazon Personalize, vous devez enregistrer les horodatages au format [Unix Epoch](https://en.wikipedia.org/wiki/Unix_time).

Actuellement, les valeurs d'horodatage ne sont pas lisibles par l'homme. Prenons donc une valeur d'horodatage arbitraire et voyons comment l'interpréter.

In [ ]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

Oups ! Pour cette valeur d'horodatage particulière, le code a rendu l'année 41 132. C'est un peu loin dans le futur pour nous, et, clairement, ce n'est la bonne façon d'analyser les données. Nous avons besoin d'une deuxième tentative.

JavaScript enregistre le temps en millisecondes, et il s'agit d'une collection de données provenant d'une application web. Divisons donc la valeur de l'horodatage par 1 000 avant d'appliquer notre code.

In [ ]:
arb_time_stamp = arb_time_stamp/1000
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

Le mois de février 2009 semble beaucoup plus réaliste pour notre jeu de données. Nous n'avons pas besoin d'horodatages lisible par l'homme pour utiliser Amazon Personalize, mais nous tenons à ce que les dates soient réalistes. Nous allons donc transformer chaque horodatage du jeu de données pour l'éloigner du format JavaScript en millisecondes. 

In [ ]:
original_data.timestamp = original_data.timestamp / 1000
original_data.head(5)

Effectuez un test d'intégrité du jeu de données transformé en choisissant un horodatage arbitraire et en le transformant en un format lisible par l'homme.

In [ ]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

Cette date a du sens en tant qu'horodatage. Nous pouvons donc continuer à formater le reste des données. Rappelez-vous que les données dont nous avons besoin sont des données d'interaction utilisateur-article, qui sont `userID`, `artistID` et `timestamp` dans ce cas. Notre jeu de données comporte une colonne supplémentaire, `tagID`, qui peut être supprimée du jeu de données.

In [ ]:
interactions_df = original_data.copy()
interactions_df = interactions_df[['userID', 'artistID', 'timestamp']]
interactions_df.head()

Après avoir manipulé les données, vérifiez toujours si le format des données a changé.

In [ ]:
interactions_df.dtypes

Dans ce cas, le format int64 de la colonne horodatage a été remplacé par float64. Par conséquent, rétablissons le format int64.

In [ ]:
interactions_df.astype({'timestamp': 'int64'}).dtypes

 Amazon Personalize utilise des noms de colonne par défaut pour les utilisateurs, les articles et l'horodatage. Ces noms de colonnes par défaut sont `USER_ID`, `ITEM_ID` ET `TIMESTAMP`. La dernière modification apportée au jeu de données consiste donc à remplacer les en-têtes de colonnes existants par les en-têtes par défaut.

In [ ]:
interactions_df.rename(columns = {'userID':'USER_ID', 'artistID':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

Voilà ! À ce stade, les données sont prêtes à être utilisées et il ne nous reste plus qu'à les enregistrer dans un fichier CSV.

In [ ]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## Créer des groupes de jeux de données et le jeu de données d'interactions <a class="anchor" id="group_dataset"></a>
[Retour au début](#top)

Le plus haut niveau d'isolation et d'abstraction avec Amazon Personalize est le *groupe de jeux de données*. Toute information stockée dans l'un de ces groupes de jeux de données n'a aucun impact sur un autre groupe de données ou sur les modèles créés à partir de celui-ci – ils sont entièrement indépendants. Vous pouvez ainsi réaliser de nombreuses expériences, et c'est en partie grâce à cela que vos modèles restent privés et ne sont entraînés que sur vos données. 

Avant d'importer les données préparées antérieurement, il faut créer un groupe de jeux de données et y ajouter un jeu de données qui gère les interactions.

Les groupes de jeux de données peuvent comporter les types d'informations suivants :

* Interactions utilisateur-article
* Flux d'événements (interactions en temps réel)
* Métadonnées de l'utilisateur
* Métadonnées de l'article

Avant de créer le groupe de jeux de données et le jeu de données pour nos données d'interaction, vérifions que votre environnement peut communiquer avec Amazon Personalize.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Créer le groupe de jeux de données

La cellule suivante va créer un nouveau groupe de jeux de données avec le nom `personalize-poc-lastfm`.

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-batch-recommendations-dg"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

Pour pouvoir utiliser le groupe de jeux de données, il doit être actif. Cela peut prendre une ou deux minutes. Exécutez la cellule ci-dessous et attendez qu'elle affiche le statut ACTIF. Il vérifie l'état du groupe de jeux de données toutes les secondes, jusqu'à un maximum de 3 heures.

In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Maintenant que vous avez un groupe de jeux de données, vous pouvez créer un jeu de données pour les données d'interaction.

### Créer le jeu de données

Tout d'abord, définissez un schéma afin d'indiquer à Amazon Personalize le type de jeu de données que vous chargez. Plusieurs mots-clés réservés et obligatoires sont requis dans le schéma, en fonction du type de jeu de données. Des informations plus détaillées peuvent être retrouvées dans la [documentation](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Ici, vous allez créer un schéma pour les données d'interactions, qui nécessite les champs `USER_ID`, `ITEM_ID` et `TIMESTAMP`  Ils doivent être définis dans le schéma en respectant l'ordre dans lequel ils apparaissent dans le jeu de données.

In [ ]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-batch-recommendations-interactions",
    schema = json.dumps(interactions_schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

Une fois le schéma créé, vous pouvez créer un jeu de données dans le groupe de jeux de données. Il est à noter que cette opération ne charge pas encore les données. Cela arrivera quelques étapes plus tard.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-batch-recommendations-ds",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

## Configurer un compartiment S3 et un rôle IAM <a class="anchor" id="bucket_role"></a>
[Retour au début](#top)

Pour l'instant, nous avons téléchargé, manipulé et enregistré les données sur l'instance Amazon EBS associée à l'instance qui exécute ce bloc-notes Jupyter. Toutefois, Amazon Personalize aura besoin d'un compartiment S3 pour servir de source à vos données, ainsi que de rôles IAM pour accéder à ce compartiment. Mettons tout cela en place.

Utilisez les métadonnées stockées sur l'instance sous-jacente de ce bloc-notes Amazon SageMaker pour déterminer la région dans laquelle il fonctionne. Si vous utilisez un bloc-notes Jupyter en dehors d'Amazon SageMaker, définissez simplement la région sous forme de chaîne comme suit. Le compartiment Amazon S3 doit se trouver dans la même région que les ressources Amazon Personalize que nous avons créées jusqu'à présent.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

Les noms des compartiments Amazon S3 sont uniques au niveau mondial. Pour créer un nom de compartiment unique, le code ci-dessous ajoutera la chaîne de caractères `personalizepoc` à votre numéro de compte AWS. Puis il crée un compartiment avec ce nom dans la région découverte dans la cellule précédente.

In [ ]:
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-personalize-batch-recommendations"
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

### Charger les données vers S3

Maintenant que votre compartiment Amazon S3 a été créé, chargez le fichier CSV de nos données d'interaction utilisateur-article. 

In [ ]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### Définir la politique du compartiment S3
Amazon Personalize doit pouvoir lire le contenu de votre compartiment S3. Il faut donc ajouter une politique de compartiment qui l'autorise.

In [ ]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

### Créer un rôle IAM

Amazon Personalize doit pouvoir assumer des rôles dans AWS, afin de disposer des autorisations nécessaires pour exécuter certaines tâches. Créons un rôle IAM et ajoutons-lui les politiques requises. Le code ci-dessous associe des politiques très permissives ; veuillez utiliser des politiques plus restrictives pour toute application de production.

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleBatchRecommendations"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

## Importer les données d'interactions <a class="anchor" id="import"></a>
[Retour au début](#top)

Précédemment, vous avez créé le groupe de jeux de données et le jeu de données pour héberger vos informations. Vous allez maintenant exécuter une tâche d'importation qui chargera les données du compartiment S3 dans le jeu de données Amazon Personalize. 

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-batch-recommendations",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

Pour pouvoir utiliser le jeu de données, la tâche d'importation doit être active. Exécutez la cellule ci-dessous et attendez qu'elle affiche le statut ACTIF. Il vérifie l'état de la tâche d'importation toutes les secondes, jusqu'à un maximum de 3 heures.

L'importation des données peut prendre un certain temps, en fonction de la taille du jeu de données. Dans cet atelier, l'importation des données devrait durer environ 15 minutes.

In [ ]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Lorsque l'importation du jeu de données est active, vous êtes prêt à commencer à créer des modèles avec SIMS, Classement personnalisé, Compte de popularité et Personnalisation de l'utilisateur AWS. Ce processus se poursuivra dans d'autres blocs-notes. Exécutez la cellule ci-dessous avant de passer au stockage de quelques valeurs à utiliser dans les prochains blocs-notes.

## Créer des solutions <a class="anchor" id="solutions"></a>
[Retour au début](#top)

Dans ce bloc-notes, vous allez créer des solutions avec la recette suivante :

1. Personnalisation de l'utilisateur AWS


Dans Amazon Personalize, une variation spécifique d'un algorithme est appelée une recette. Des recettes différentes conviennent à des situations différentes. Un modèle entraîné s'appelle une solution, et chaque solution peut avoir de nombreuses versions qui se rapportent à un volume de données spécifique au moment où le modèle a été entraîné.

Tout d'abord, nous allons énumérer toutes les recettes qui sont prises en charge. Cela vous permettra d'en sélectionner une et de l'utiliser pour créer votre modèle.

In [ ]:
personalize.list_recipes()

Le résultat est juste une représentation JSON de tous les algorithmes mentionnés dans l'introduction.

Ensuite, nous sélectionnerons des recettes spécifiques et créerons des modèles avec elles.

### Personnalisation de l'utilisateur AWS

La personnalisation de l'utilisateur AWS est l'un des modèles de recommandation les plus avancés disponibles. Elle permet de mettre à jour les recommandations en temps réel en fonction du comportement des utilisateurs. Elle tend également à surpasser d'autres approches, comme le filtrage collaboratif. Cette recette est la plus longue à entraîner. Commençons par elle.

Dans notre cas d'utilisation, en utilisant les données LastFM, nous pouvons utiliser l'algorithme de personnalisation de l'utilisateur AWS pour recommander de nouveaux artistes à un utilisateur en fonction de son comportement antérieur d'identification des artistes. Rappelez-vous que nous avons utilisé les données de marquage pour représenter les interactions positives entre un utilisateur et un artiste.

Tout d'abord, sélectionnez la recette en trouvant l'ARN dans la liste des recettes ci-dessus.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

#### Créer la solution

Tout d'abord, vous créez une solution à l'aide de la recette. Même si vous fournissez le jeu de données ARN dans cette étape, le modèle n'est pas encore entraîné. Considérez cela comme un identifiant plutôt qu'un modèle entraîné.

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-batch-recommendations-user-personalization",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Créer la version de solution

Une fois que vous avez une solution, vous devez créer une version afin d'achever l'entraînement du modèle. L'entraînement peut prendre un certain temps, plus de 25 minutes, et une moyenne de 40 minutes pour cette recette avec notre jeu de données. Normalement, nous utiliserions une boucle while pour interroger le système jusqu'à ce que la tâche soit terminée. Toutefois, cette tâche bloquerait l'exécution d'autres cellules, et l'objectif ici est de créer de nombreux modèles et de les déployer rapidement. Nous allons donc mettre en place la boucle while pour toutes les solutions plus bas dans ce bloc-notes. Vous y trouverez également des instructions pour visualiser la progression dans la console AWS.

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

In [ ]:
solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

### Affichez le statut de création de la solution

Voici, comme promis, comment afficher les mises à jour du statut dans la console :

* La console AWS devrait déjà être ouverte dans un autre onglet du navigateur, après l'ouverture de cette instance du bloc-notes. 
* Passez à cet onglet et recherchez en haut le service `Personalize`, puis référez-vous à cette page de service. 
* Cliquez sur `View dataset groups`.
* Cliquez sur le nom de votre groupe de jeux de données, très probablement un nom contenant POC.
* Cliquez sur `Solutions and recipes`.
* Vous voyez maintenant la liste de toutes les solutions que vous avez créées ci-dessus, y compris une colonne avec le statut des versions de la solution. Une fois qu'elle est `Active`, votre solution est prête à être examinée. Elle est également en mesure d'être déployée.

Ou exécutez simplement la cellule ci-dessous pour surveiller le statut de création des versions de la solution.

In [ ]:
in_progress_solution_versions = [
    solution_version_arn,
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

## Interagir avec les solutions <a class="anchor" id="interact"></a>
[Retour au début](#top)

Maintenant que toutes nos solutions sont déployées et actives, nous pouvons commencer à obtenir des recommandations au moyen d'un appel d'API. 

Commencez par charger le jeu de données que nous pouvons utiliser pour notre table de consultation.

In [ ]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/artists.dat', delimiter='\t', index_col=0)

# Render some sample data
items_df.head(5)

En définissant la colonne ID comme colonne d'index, il est aisé de trouver un artiste en recherchant simplement l'ID.

In [ ]:
item_id_example = 987
artist = items_df.loc[item_id_example]['name']
print(artist)

Cette méthode est acceptable. Cependant, il serait fastidieux de la répéter partout dans notre code. C'est pourquoi la fonction ci-dessous va la supprimer.

In [ ]:
def get_artist_by_id(artist_id, artist_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return artist_df.loc[int(artist_id)]['name']
    except:
        return "Error obtaining artist"

Testons à présent quelques valeurs simples pour vérifier notre système de détection d'erreurs.

In [ ]:
# A known good id
print(get_artist_by_id(artist_id="987"))
# A bad type of value
print(get_artist_by_id(artist_id="987.9393939"))
# Really bad values
print(get_artist_by_id(artist_id="Steve"))

Super ! Nous disposons désormais d'un moyen de présentation des résultats. 

## Recommandations par lots<a class="anchor" id="batch"></a>
[Retour au début](#top)

Dans de nombreux cas, vous pouvez souhaiter disposer d'un plus grand jeu de données de recommandations exportées. Récemment, Amazon Personalize a introduit les recommandations par lots comme moyen d'exporter une collection de recommandations vers S3. Dans cet exemple, nous allons voir comment procéder pour la solution de personnalisation de l'utilisateur AWS. Pour en savoir plus sur les recommandations par lots, veuillez vous reporter à la [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). Cette fonction s'applique à toutes les recettes, mais le format de sortie varie.

Une implémentation simple se présente comme suit :

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "ARN de la version de la solution,"
    jobName = "Nom de la tâche du lot",
    roleArn = "ARN du rôle IAM",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

L'importation du kit SDK, l'ARN de la version de la solution et les ARN de rôle ont tous été définis. Il ne reste plus qu'à définir une entrée, une sortie et un nom de tâche.

En commençant par l'entrée pour la personnalisation de l'utilisateur, cela se présente comme suit :


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

Le résultat devrait ressembler à ce qui suit :

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

La sortie est un fichier JSON Lines. Il est constitué d'objets JSON individuels, un par ligne. Nous devrons donc fournir un effort supplémentaire par la suite pour digérer les résultats dans ce format.

### Création du fichier d'entrée

Lorsque vous utilisez la fonction de traitement par lots, vous indiquez les utilisateurs pour lesquels vous souhaitez recevoir des recommandations lorsque la tâche est terminée. La cellule ci-dessous va à nouveau sélectionner quelques utilisateurs au hasard, puis créer le fichier et le sauvegarder sur le disque. À partir de là, vous le chargerez sur S3 pour l'utiliser plus tard dans l'appel d'API.

In [ ]:
users_df = pd.read_csv(data_dir + '/user_artists.dat', delimiter='\t', index_col=0)
# Render some sample data
users_df.head(5)

In [ ]:
# Get the user list
batch_users = users_df.sample(3).index.tolist()

# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in batch_users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [ ]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

Chargez le fichier vers S3 et enregistrez le chemin d'accès comme variable pour plus tard.

In [ ]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

Les recommandations par lots lisent l'entrée du fichier que nous avons chargé sur S3. De la même manière, les recommandations par lots enregistreront le résultat dans un fichier dans S3. Nous définissons donc le chemin de sortie où les résultats doivent être enregistrés.

In [ ]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

Maintenant, exécutez simplement l'appel pour lancer l'exportation par lots.

In [ ]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = solution_version_arn,
    jobName = "Batch-Inference-Job",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Exécutez la boucle while ci-dessous pour suivre le statut de l'appel de recommandation par lot. Cette opération peut prendre environ 25 minutes, car Personalize doit mettre en place l'infrastructure nécessaire à l'exécution de cette tâche. Nous testons la fonction avec un jeu de données de seulement 3 utilisateurs, ce qui n'est pas une utilisation efficace de ce mécanisme. Normalement, vous n'utiliserez cette fonction que pour le traitement en masse, auquel cas les gains d'efficacité deviendront évidents.

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Une fois que la tâche des recommandations par lots a terminé le traitement, nous pouvons analyser la sortie chargée vers S3.

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            artist = get_artist_by_id(item)
            recommendation_list.append(artist)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## Nettoyer les solutions

Nettoyez ensuite les solutions. Le code ci-dessous énumérera toutes les solutions dans votre compte.

In [ ]:
paginator = personalize.get_paginator('list_solutions')
for paginate_result in paginator.paginate():
    for solution in paginate_result["solutions"]:
        print(solution["solutionArn"])

Consultez la liste des ARN pour déterminer la solution à supprimer. Utilisez ensuite le code ci-dessous pour le supprimer en insérant l'ARN.

In [ ]:
personalize.delete_solution(
    solutionArn = "INSERT ARN HERE"
)

## Nettoyer les jeux de données

Nettoyez ensuite les jeux de données. Le code ci-dessous énumérera tous les jeux de données de votre compte.

In [ ]:
paginator = personalize.get_paginator('list_datasets')
for paginate_result in paginator.paginate():
    for datasets in paginate_result["datasets"]:
        print(datasets["datasetArn"])

Consultez la liste des ARN pour déterminer le jeu de données à supprimer. Utilisez ensuite le code ci-dessous pour le supprimer en insérant l'ARN.

In [ ]:
personalize.delete_dataset(
    datasetArn = "INSERT ARN HERE"
)

## Nettoyer les schémas

Nettoyez ensuite les schémas. Le code ci-dessous énumérera tous les schémas de votre compte.

In [ ]:
paginator = personalize.get_paginator('list_schemas')
for paginate_result in paginator.paginate():
    for schema in paginate_result["schemas"]:
        print(schema["schemaArn"])

Consultez la liste des ARN pour déterminer le schéma à supprimer. Utilisez ensuite le code ci-dessous pour le supprimer en insérant l'ARN.

In [ ]:
personalize.delete_schema(
    schemaArn = "INSERT ARN HERE"
)

## Nettoyer le groupe de jeux de données

Enfin, nettoyez le groupe de jeux de données. Le code ci-dessous énumérera tous les groupes de jeux de données de votre compte.

In [ ]:
paginator = personalize.get_paginator('list_dataset_groups')
for paginate_result in paginator.paginate():
    for dataset_group in paginate_result["datasetGroups"]:
        print(dataset_group["datasetGroupArn"])

Consultez la liste des ARN pour déterminer le groupe de jeux de données à supprimer. Utilisez ensuite le code ci-dessous pour le supprimer en insérant l'ARN.

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = "INSERT ARN HERE"
)

## Nettoyer le compartiment S3 et le rôle IAM

Vous pouvez éventuellement supprimer le rôle IAM et le compartiment S3 que nous avons créés pendant l'atelier.

Commencez par répertorier tous les rôles IAM de votre compte à l'aide du code ci-dessous.

In [ ]:
iam = boto3.client('iam')

paginator = iam.get_paginator('list_roles')
for paginate_result in paginator.paginate():
    for roles in paginate_result["Roles"]:
        print(roles["RoleName"])

Identifiez le nom du rôle que vous souhaitez supprimer.

Vous ne pouvez pas supprimer un rôle IAM auquel des politiques sont encore attachées. Donc, après avoir identifié le rôle pertinent, répertorions la liste des politiques attachées au rôle.

In [ ]:
iam.list_attached_role_policies(
    RoleName = "INSERT ROLE NAME HERE"
)

Vous devez détacher les politiques dans le résultat ci-dessus en utilisant le code ci-dessous. Répétez l'opération pour chaque politique attachée.

In [ ]:
iam.detach_role_policy(
    RoleName = "INSERT ROLE NAME HERE",
    PolicyArn = "INSERT ARN HERE"
)

Enfin, vous devriez être en mesure de supprimer le rôle IAM.

In [ ]:
iam.delete_role(
    RoleName = "INSERT ROLE NAME HERE"
)

Pour supprimer un compartiment S3, il doit d'abord être vide. La façon la plus facile de supprimer un compartiment S3 est simplement de naviguer vers S3 dans la console AWS, de supprimer les objets du compartiment, puis de supprimer le compartiment S3 lui-même.